In [0]:
#dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()

In [0]:
%pip install azure-storage-blob
%pip install pandas

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-a9a54a27-81ff-44be-aefd-0e8b5e5c6e8f
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() t

In [0]:
# Restart the Python kernel to apply the changes
dbutils.library.restartPython()

In [0]:
import pandas as pd
import os
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [0]:
# Get the current working directory
os.getcwd()

'/databricks/driver'

In [0]:
#Convert multiple Parquet files to CSV files
#Connct to Azure storage account
account_name = "nycyellowtaxidataset"
account_key = "your_account_key"  # Replace with your actual storage account key
container_name = "nycyellowtaxisdatasets"

# Initialize BlobServiceClient
connection_string = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

container_client = blob_service_client.get_container_client(container_name)
blobs_list = container_client.list_blobs()

# Extract blob names from the blobs list
parquet_files = [blob.name.split('/')[1] for blob in blobs_list if blob.name.endswith(".parquet")]

# Loop through each Parquet file
for parquet_file in parquet_files:
    # Download Parquet file from Azure Storage
    parquet_blob_path = f"parquetfiles/{parquet_file}"
    local_parquet_file = f"/databricks/driver/{parquet_file}"

    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(parquet_blob_path)

    with open(local_parquet_file, "wb") as data:
        data.write(blob_client.download_blob().readall())

    # Read Parquet file into a pandas DataFrame
    parquet_df = pd.read_parquet(local_parquet_file)

    # Convert DataFrame to CSV
    csv_filename = os.path.splitext(parquet_file)[0] + ".csv"
    csv_blob_path = f"csvoutputfiles/{csv_filename}"
    parquet_df.to_csv(csv_filename, index=False)

    # Upload CSV file to Azure Storage
    blob_client = container_client.get_blob_client(csv_blob_path)
    with open(csv_filename, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)

    print(f"Conversion successful for {parquet_file}")

Conversion successful for yellow_tripdata_2023-07.parquet
Conversion successful for yellow_tripdata_2023-08.parquet
Conversion successful for yellow_tripdata_2023-09.parquet
